# Reverse Engineering MechWarrior 3 Assets

The aim of this project is to be able to extract most MechWarrior 3 assets to modern, well-documented, and usable formats, for preservation. This is a journal of how I reverse engineered (RE) these assets. I've released functioning code in the [MechWarrior 3 Asset Extractor](https://github.com/tobywf/mech3ax) repository.

## Status

* Ambient tracks: Semi-automated with robust, multi-platform options (not via mech3ax)
* Videos: Semi-automated with robust, multi-platform options (not via mech3ax)
* Sounds: Fully automated, cross platform
* Textures: Fully automated, cross platform
* 3D models: Fully automated, cross platform
* Animations: Fully automated, cross platform. Some issues remaining when applying the animations to models
* Game/campaign data: Fully automated, cross platform. Not much use for now
* Executables: Fully automated extraction of message data from `Mech3Msg.dll`, largely understood. `Mech3.exe` is poorly understood. My focus is on assets, not game engine code.
* Maps and interpreter instructions: Little progress. Some data can be extracted to JSON
* Pirate's Moon: Not started

## Project structure

This notebook provides a basic introduction.

In notebooks 02 and 03, I'll look at media obviously present on the disk. After that, I'll install the different version and compare the resulting installation files. Then only does it make sense to tackle reverse engineering (RE) the different archives. Sounds are up first, since they turned out to be easiest. Next are textures. Their visual nature also makes it easy to RE without diving into the executable. For the models, this required significant RE of the main executable and is somewhat opaque, but also the most rewarding when a model can finally be rendered. Motion/animations also was not easy, but mainly trial and error. Finally, the game's supporting files. I made some progress on these, but without a game engine, they're just files with information.

## A note on failure

Please don't be too impressed or intimidated by such a project. I've been working on/off for several years, and never thought I'd get this far. It was mainly perseverance and not skill.

What you're seeing here is a highly edited version of the process I went through, edited with the benefit of hindsight. Hindsight is huge when reverse engineering, and lets you see many more connections than when you're sitting there, starting from scratch. While I've tried to keep a lot of the details in on how I got stuff working, to keep it readable many false starts and wrong avenues have been edited out.

Without failing, you can't succeed. I think this is called learning, or what learning should be (unlike sometimes in school where it's just memorisation). And that was the main goal of this undertaking. So I hope this motivates you. If you don't know how to do something, that is the point.

## Introduction

### MechWarrior history

If you want a more entertaining history, Chase "Scharmers" Dahl has an awesome review called [Fifteen Years of Giant Robots](https://www.quartertothree.com/features/robots/robots_1.shtml#intro). Or - if video is your thing - check out The Examined Life (of Gaming)'s [MechWarrior Retrospective](https://www.youtube.com/watch?list=PLzqQqTZGYi1ZqlVw-_B0mdPOMU3NffkXs&v=WuW0UCyvisM) series (the [MechWarrior 3 episode](https://www.youtube.com/watch?v=f4a-6RIgVfA&index=5&list=PLzqQqTZGYi1ZqlVw-_B0mdPOMU3NffkXs)). I highly recommend both. Personally, I don't have much interest in the games before MechWarrior 3, because I never played them.

In what is now typical fashion, MechWarrior 3 isn't the third instalment of the MechWarrior series, but the third generation. It was published in May 1999, with a new engine (again, it's worth reading Scharmers' write-up of [the development history](https://www.quartertothree.com/features/robots/robots_4.shtml)). It received an expansion pack, Pirate's Moon, and a Gold Edition release in September 1999.

The fourth generation released quite quickly afterwards, with MechWarrior 4: Vengeance in late 2000 in North America. It was a huge jump graphically, and brought with it significant gameplay changes (e.g. weapons hardpoints). MechWarrior 4: Black Knight is the expansion pack released not quite a year later (2001), followed by the stand-alone MechWarrior 4: Mercenaries, also about a year later (2002). Interestingly, a few months before Mercenaries, the Clan and Inner Sphere 'Mech Paks were released, which provided four new 'mechs each, and some weapons (they [were panned](https://www.ign.com/articles/2002/08/29/mechwarrior-4-clan-mech-pak-review) as [a money-grab](https://www.ign.com/articles/2002/07/16/mechwarrior-4-inner-sphere-mech-pack-review)).

I believe due to issues with the rights, there was a lull in the history. Finally in 2010, MechWarrior 4: Mercenaries [was made available for free](http://www.sarna.net/wiki/Studio_MekTek,_Inc). Then, in 2013, [MechWarrior Online](https://mwomercs.com/) was released. It focused on multiplayer battles, and still has a strong community to this day. In 2018, [BattleTech](http://battletechgame.com/) was released, and although is a top-down, turn-based strategy game, it was good to see renewed development efforts with the IP (I enjoyed BattleTech a lot). Very excitingly, MechWarrior 5: Mercenaries was scheduled for release in 2018; this was pushed to 2019 and is now an Epic Store exclusive. I may sign up for an account if I can be bothered.


###  Why bother?

Speaking of bothering, MechWarrior 4 and onwards certainly offer a more balanced, tactical approach with the previously mentioned weapon hardpoints. So why this game? In my mind, none of these games came close to the campaign of MechWarrior 3. Future campaigns have you starting off as a scrappy lance, but quickly growing and often being able to pick missions for different factions - which I never ended up caring about. MechWarrior 3 is different. Nothing comes close to having to complete an entire operation that goes wrong from the start, with limited supplies, out of date tactical information. Despite the troubled development which can be felt in lacking graphics for the time, barren landscapes, lancemates you hear over the radio more often than you see them, the story shines. This is why it sticks in my head.

Seems I'm not the only one, as there are hundreds of posts trying to get it to work on modern Windows. The most promising approach is [dgVoodoo 2](http://dege.freeweb.hu/), "a wrapper for old graphics API's for Windows Vista/7/8/10". There are still issues with the physics on today's fast processors though.

My goal is to reverse engineer MechWarrior 3 (MW3) to extract all vital game assets into well-documented and understood formats for preservation. I'd love to see somebody take those assets and put them in a new game engine. That's much harder and such projects tend to fall apart. Even so, this project has already taken me several years of on/off work. But I hope that by documenting the formats and providing easy to load assets, the focus could be on a game engine/framework that wouldn't have to worry less about legal challenges if it can simply make use of common formats.

### The MechWarrior 3 community

There still exists a modding community, and people still play MW3 online. This sounds ideal! But when I reached out a few years ago, there was significant trepidation that a reverse engineering project would make cheating easier. Initially, I would have loved to build on the work of MW3 legends like Finnegan McCool. But in retrospect, I think this was a blessing in disguise. After all, part of this project is for me to learn reverse engineering. At this stage, I have not reverse engineered the maps/campaign format though, so there is still opportunity for this. Contributions are most welcome! Lastly, I hope this project doesn't confirm their fears about online cheating.

Also, thanks to [sarna.net](http://www.sarna.net) for keeping all things BattleTech around in such a wonderfully accessible way (including patches).

## Pre-requisites

[Python 3.7](https://www.python.org/) and [Jupyter Notebook](https://jupyter.org/) are the main tools I've used. In fact, almost all these steps can be done with any operating system (thanks Python). Throughout, I've noted all the tools I use, and cross-platform alternatives when I know of them. More on this in the appropriate chapters though.

I also recommend a Windows machine - a VM or anything running Windows 95 or above will do. On the CD, the data is stored as InstallShield archives. I wasted quite some time reverse engineering them, only to find a much [better project (unshield)](https://github.com/twogood/unshield) already exists. But while the InstallShield archives can be extracted, it's far easier to install MW3 and then grab the files. This is for sure the way to go, since the files end up much more logically arranged  when installed, you can select only certain features (e.g. the most high-quality textures), and patches can be applied. DRM (discussed later) is also less of a concern this way I think.

Finally, if you really want to follow along with the reverse engineering, you will need a disassembler. For basic reconnaissance, I used [Radare2](https://rada.re/). The CLI output is clean and easily shareable. For actual disassembly, I'd recommend a GUI. The [Cutter project](https://cutter.re/) is a GUI based on Radare2. The most popular disassembler is Hex-Rays' IDA , and there [exists a free version](https://www.hex-rays.com/products/ida/support/download_freeware.shtml). Finally, [NSA's Ghidra](https://ghidra-sre.org/) is very capable (apart from the Java JRE requirement and the ugly UI). The decompilation feature is particularly excellent from my experience with trying out all three, and I would recommend it for RE noobs like myself.

## MechWarrior 3 versions

In the US, there seem to have been three releases: version 1.0, 1.1, and Gold Edition.  At least I've managed to acquire those. They can all be patched to 1.2. Presumably there was also a 1.1 patch (which I have not been able to find).

In a weird quirk, the Gold Edition Readme says it is version 1.2, but it is still missing two multiplayer maps, `zbd/c3/readermp3.zbd` and `zbd/c3/readermp4.zbd`. Applying the 1.2 patch will install these.

The game was localised for German and French. I only have the 1.0 German version, and couldn't get hold of a French version. Please do reach out if you have a version I'm missing. I would love to confirm the code works for all versions.

The German localisation is outstanding. Everything was localised, including the intro cinematic, the mission briefing, and in-game dialog. This was a huge selling point for me at the time (my English wasn't quite as polished yet), along with the kick-ass box art (seriously, that Mad Cat). The German CD cover is also gorgeous. Apparently though, the German version [was censored](https://pcgamingwiki.com/wiki/MechWarrior_3#Localizations). This rings true, as Germany has always had strict rules for video games certification via the USK and JuSchG. For example, the terrorists in "Command &amp; Conquer: Generals" were replaced by robots. The gibs are quite gruesome when stomping on infantry, and seem largely unnecessary in a 'mech-focussed game.

## DRM

The PC Gaming Wiki claims [MechWarrior 3 is protected by Macrovision's SafeDisc DRM](https://pcgamingwiki.com/wiki/MechWarrior_3). At the time MW3 was released, only SafeDisk version 1 was available. Instructions from [CD Media World](https://www.cdmediaworld.com/hardware/cdrom/cd_protections_safedisc.shtml) on how to detect SafeDisc protection:

> The following files should exist on every the original CD: **00000001.TMP**, **CLCD16.DLL**, **CLCD32.DLL**, **CLOKSPL.EXE**, **DPLAYERX.DLL**

> There is always a **GAME.EXE** and **GAME.ICD** file where the **.ICD** is the original game executable (in encrypted form) and the **.EXE** is a loader containing a parts of the **SafeDisc** protection.

(Formatting edited for readability.) The [Wine mailing list agrees](https://www.winehq.org/pipermail/wine-users/2002-April/007910.html) largely, sometimes `SECDRV.SYS` and `DRVMGT.DLL` are also found. 

None of the US version I own have any of these files, the German version does though. It is possible the US versions have an earlier variant of SafeDisc copy protection, based on the earlier SafeAudio copy protection It uses weak sectors to detect when a disk has been copied. (For more information, see [this CD Freaks/Myce article](https://www.myce.com/article/SafeDisc-2-Explained-and-Defeated___-181/) on SafeDisc 2.)

There are indications something odd is present on US disks. When I list the `video` directory, the date of the parent directory (`..`) is always mangled:


Version 1.00 (Deutsch):

```
04/06/1999  02:25    <DIR>          .
04/06/1999  02:25    <DIR>          ..
```

Version 1.00 (U.S.):

```
12/05/1999  02:18    <DIR>          .
The parameter is incorrect.
<0x16>?      <DIR>          ..
```

Version 1.1 (U.S.):

```
09/07/1999  12:01    <DIR>          .
The parameter is incorrect.

?      <DIR>                              ..
```

Version 1.2 (U.S.):

```
05/10/1999  08:35    <DIR>          .
<0x11>?      <DIR>          ..
```

SafeDisc itself is a liability, as the driver [contains a buffer overflow vulnerability (CVE-2007-5587)](https://nvd.nist.gov/vuln/detail/CVE-2007-5587).

I don't want to comment too much on DRM, although as a customer, it has always been an annoyance and a hindrance for me. But it is a concern for any reverse engineering effort. Some countries allow circumventing DRM for abandoned products or legitimate fair use. Some don't. This is why I've approached the project by installing the game, and then working on binary files as much as possible. No DRM is bypassed.

## Next up

[Ambient tracks](02-ambient-tracks.ipynb)